In [ ]:
from agentor.memory.api import Memory

In [ ]:
mem = Memory()

to_process = {
    "user": "How many 'r's in strawberries?",
    "agent": "there are 0 'r's in strawberries",
}
mem.add(to_process)
mem.get_full_conversation()

In [ ]:
to_process = {
    "user": "What is the capital of France?",
    "agent": "Paris is the capital of France",
}
mem.add(to_process)
mem.get_full_conversation()

In [ ]:
mem.search("capital of a city", limit=5)

In [ ]:
# from openai import OpenAI
#
# client = OpenAI()
#
# response = client.responses.create(
#     model="gpt-5-mini",
#     input="how many 'r's in Strawberries",
#     reasoning={"effort": "low"},
# )

In [ ]:
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

db = lancedb.connect("/tmp/db")
model = (
    get_registry()
    .get("sentence-transformers")
    .create(name="google/embeddinggemma-300M", device="cpu")
)


class Words(LanceModel):
    text: str = model.SourceField()
    vector: Vector(model.ndims()) = model.VectorField()


table = db.create_table("words", schema=Words)
table.add([{"text": "hello world"}, {"text": "goodbye world"}])

query = "greetings"
actual = table.search(query).limit(1).to_pydantic(Words)[0]
print(actual.text)

In [ ]:
from agents import Agent
from agentor.memory.tools import memory_search, memory_get_full_conversation, memory_add

_instructions = """
You are a memory agent. You are responsible for finding relevent information from the memory for the user's request and updating the memory with the new information.

You have the following tools:
- memory_search tool to search the memory for the most relevant conversations.
- memory_get_full_conversation tool to get the full conversation from the memory.
- memory_add tool to add a conversation to the memory.
"""


def build_memory_agent(model: str = "gpt-5-mini") -> Agent:
    return Agent(
        name="Memory agent",
        instructions=_instructions,
        tools=[memory_search, memory_get_full_conversation, memory_add],
        model=model,
    )


from agents import Runner, RunContextWrapper
from agentor.utils import AppContext
from agentor.memory.api import Memory

In [ ]:
mem = Memory()

In [ ]:
agent = build_memory_agent()
runner = Runner()

RunResult:
- Last agent: Agent(name="Memory agent", ...)
- Final output (str):
    The capital of France is Paris.
    
    Note: I tried to check and update your memory but couldn't access the memory service right now. Would you like me to try again?
- 11 new item(s)
- 4 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [ ]:
from agentor.utils import CoreServices

output = runner.run_streamed(
    agent,
    "What is the capital of France?",
    context=RunContextWrapper(AppContext(core=CoreServices(memory=mem))),
)
async for chunk in output.stream_events():
    print(chunk)